### Info
based on notebook3\
conversion of qkeras to hls4ml

### Load packages and the model

In [1]:
import tensorflow as tf
import numpy
import h5py
from qkeras.utils import _add_supported_quantized_objects
from qkeras import quantized_bits
co = {}; _add_supported_quantized_objects(co)

model_path = f"/eos/project/c/cms-l1ml/public/Pawel/axol1tl_model.h5"

model = tf.keras.models.load_model(model_path, custom_objects=co)
#model_config = model.get_config()

data_path = f"/eos/project/c/cms-l1ml/public/Pawel/complete.h5"
data_file = h5py.File(data_path, 'r')

2025-10-02 17:25:07.761281: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-02 17:25:08.039503: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-10-02 17:25:08.039559: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-10-02 17:25:08.039613: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-02 17:25:08.088385: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-02 17:25:08.088902: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

### Copy weights

In [2]:
ds_weights = data_file['model']['trimmed_encoder']['model_weights']
for layer in iter(model.layers):
    #print(layer.name)
    if layer.name in ('input_1','subtract','dot'): continue

    if layer.name == 'q_dense_4':
        weights_group = ds_weights['q_dense_4']['variational_auto_encoder']['vae__encoder']['q_dense_4']
        layer.set_weights((weights_group['kernel:0'], weights_group['bias:0']))
        continue

    weights_group = ds_weights[layer.name][layer.name]
    layer.set_weights((weights_group['kernel:0'], weights_group['bias:0']))

### Into hls4ml

In [3]:
import hls4ml

hls_config = hls4ml.utils.config_from_keras_model(
    model,
    granularity='name', 
    default_precision='fixed<18,8>',    # like the activation
    backend='Vitis')

/afs/cern.ch/user/p/psajdak/eos_mine/project_CERN/venv3_9/lib64/python3.9/site-packages/hls4ml/converters/__init__.py:27: UserWarning: WARNING: Pytorch converter is not enabled!
  warnings.warn("WARNING: Pytorch converter is not enabled!", stacklevel=1)


Interpreting Model
Topology:
Layer name: input_1, layer type: InputLayer, input shapes: [[None, 57]], output shape: [None, 57]
Layer name: q_dense, layer type: QDense, input shapes: [[None, 57]], output shape: [None, 29]
Layer name: q_dense_1, layer type: QDense, input shapes: [[None, 29]], output shape: [None, 10]
Layer name: q_dense_2, layer type: QDense, input shapes: [[None, 10]], output shape: [None, 9]
Layer name: q_dense_3, layer type: QDense, input shapes: [[None, 9]], output shape: [None, 6]
Layer name: q_dense_4, layer type: QDense, input shapes: [[None, 6]], output shape: [None, 4]
Layer name: q_dense_6, layer type: QDense, input shapes: [[None, 4]], output shape: [None, 6]
Layer name: q_dense_7, layer type: QDense, input shapes: [[None, 6]], output shape: [None, 9]
Layer name: q_dense_8, layer type: QDense, input shapes: [[None, 9]], output shape: [None, 10]
Layer name: subtract, layer type: Merge, input shapes: [[None, 10], [None, 10]], output shape: [None, 10]
Layer name:

/afs/cern.ch/user/p/psajdak/eos_mine/project_CERN/venv3_9/lib64/python3.9/site-packages/keras/src/constraints.py:365: UserWarning: The `keras.constraints.serialize()` API should only be used for objects of type `keras.constraints.Constraint`. Found an instance of type <class 'qkeras.quantizers.quantized_bits'>, which may lead to improper serialization.
  warnings.warn(


In [4]:
hls_config

{'Model': {'Precision': 'fixed<18,8>',
  'ReuseFactor': 1,
  'Strategy': 'Latency',
  'BramFactor': 1000000000,
  'TraceOutput': False},
 'LayerName': {'input_1': {'Trace': False,
   'Precision': {'result': 'fixed<18,8>'}},
  'q_dense': {'Trace': False,
   'Precision': {'result': 'fixed<18,8>',
    'weight': 'fixed<6,3>',
    'bias': 'fixed<10,7>',
    'accum': 'fixed<18,8>'},
   'ReuseFactor': 1},
  'q_dense_quantized_relu': {'Trace': False,
   'Precision': {'result': 'ufixed<10,6,RND_CONV,SAT>',
    'table': 'fixed<18,8>'},
   'ReuseFactor': 1,
   'TableSize': 1024},
  'q_dense_1': {'Trace': False,
   'Precision': {'result': 'fixed<18,8>',
    'weight': 'fixed<6,3>',
    'bias': 'fixed<10,7>',
    'accum': 'fixed<18,8>'},
   'ReuseFactor': 1},
  'q_dense_1_quantized_relu': {'Trace': False,
   'Precision': {'result': 'ufixed<10,6,RND_CONV,SAT>',
    'table': 'fixed<18,8>'},
   'ReuseFactor': 1,
   'TableSize': 1024},
  'q_dense_2': {'Trace': False,
   'Precision': {'result': 'fixed<18

For qrelu:\
'Precision': {'result': 'ufixed<10,6,RND_CONV,SAT>', 'table': 'fixed<18,8>'},\
'ReuseFactor': 1, 'TableSize': 1024

In [5]:
hls_model = hls4ml.converters.convert_from_keras_model(
    model, hls_config=hls_config, backend='Vitis',
    output_dir='./new_hls_qkeras'
)

Interpreting Model
Topology:
Layer name: input_1, layer type: InputLayer, input shapes: [[None, 57]], output shape: [None, 57]
Layer name: q_dense, layer type: QDense, input shapes: [[None, 57]], output shape: [None, 29]
Layer name: q_dense_1, layer type: QDense, input shapes: [[None, 29]], output shape: [None, 10]
Layer name: q_dense_2, layer type: QDense, input shapes: [[None, 10]], output shape: [None, 9]
Layer name: q_dense_3, layer type: QDense, input shapes: [[None, 9]], output shape: [None, 6]
Layer name: q_dense_4, layer type: QDense, input shapes: [[None, 6]], output shape: [None, 4]
Layer name: q_dense_6, layer type: QDense, input shapes: [[None, 4]], output shape: [None, 6]
Layer name: q_dense_7, layer type: QDense, input shapes: [[None, 6]], output shape: [None, 9]
Layer name: q_dense_8, layer type: QDense, input shapes: [[None, 9]], output shape: [None, 10]
Layer name: subtract, layer type: Merge, input shapes: [[None, 10], [None, 10]], output shape: [None, 10]
Layer name:

In [6]:
hls_model.write()

Writing HLS project


/afs/cern.ch/user/p/psajdak/eos_mine/project_CERN/venv3_9/lib64/python3.9/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Done


In [11]:
hls_model.compile()

Writing HLS project


Done


In [7]:
hls4ml.utils.serialization.serialize_model(hls_model,'./ser_model')

AttributeError: module 'hls4ml.utils' has no attribute 'serialization'

### Test the models

In [8]:
input_quantizer = quantized_bits(14, 6, alpha=1)
output_quantizer = quantized_bits(18,14,alpha=1)

In [9]:
X = data_file['data']["Background_data"]["Test"]["DATA"][:2048]
X_flat = X.reshape(X.shape[0],-1)

In [12]:
hls_pred = []
for i in range(0,X_flat.shape[0]):
    hls_pred_ = hls_model.predict(input_quantizer(X_flat[i]).numpy())
    hls_pred.append(output_quantizer(hls_pred_).numpy())

In [13]:
hls_pred = numpy.stack(hls_pred)

In [ ]:
qk_pred_ = model.predict(input_quantizer(X_flat[:]))

In [ ]:
qk_pred = output_quantizer(qk_pred_)
qk_pred = qk_pred.numpy()

In [14]:
print((qk_pred - hls_pred).flatten()[:1000])

NameError: name 'qk_pred' is not defined